<a href="https://colab.research.google.com/github/subhobrata/NLP_Exercises/blob/master/Week_02_Word_Embeddings_(Part_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O quora.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1ERtxpdWOgGQ3HOigqAMHTJjmOE_tWvoF"
!unzip quora.zip
!pip install -q --upgrade nltk gensim bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

Archive:  quora.zip
  inflating: train.csv               
    100% |████████████████████████████████| 1.4MB 14.9MB/s 
    100% |████████████████████████████████| 24.2MB 1.5MB/s 
    100% |████████████████████████████████| 17.5MB 2.3MB/s 
    100% |████████████████████████████████| 10.1MB 3.7MB/s 
fastai 1.0.51 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


# Word embeddings

* Nb. The notebook is heavily based on [the first assignment from the relevant ShAD course] (https://github.com/yandexdataschool/nlp_course/tree/master/week1_embeddings). *

Everyone saw these pictures (I hope):
! [embeddings relations] (https://www.tensorflow.org/images/linear-relationships.png)
* From [Vector Representations of Words, Tensorflow tutorial] (https://www.tensorflow.org/tutorials/representation/word2vec) *

Today we will deal with such models.

Let's start the morning with visualizations. We go to the site [http://rusvectores.org/ru/ru/ruhhhtpp/rusvectores.org/ru/) and see what the trained models for Russian can do.

Note the sections * Related words * and * Calculator *, as well as the set of models that you can choose from.


## We train a simple model


It’s just not fun to just look at other people's models, so we’ll gradually come closer to solving a specific task: [Quora Question Pairs at kaggle](https://www.kaggle.com/c/quora-question-pairs):

In [2]:
import pandas as pd

quora_data = pd.read_csv('train.csv')

quora_data

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


I will teach on these texts Word2vec from gensim.

First, combine all the texts.

First, combine all the texts.

In [3]:
import numpy as np

quora_data.question1 = quora_data.question1.replace(np.nan, '', regex=True)
quora_data.question2 = quora_data.question2.replace(np.nan, '', regex=True)

texts = list(pd.concat([quora_data.question1, quora_data.question2]).unique())
texts[:10]

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?',
 'Why am I mentally very lonely? How can I solve it?',
 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
 'Should I buy tiago?',
 'How can I be a good geologist?',
 'When do you use シ instead of し?',
 'Motorola (company): Can I hack my Charter Motorolla DCX3400?']


For tokenization, the easiest way is to use `nltk` (it is faster than` spacy`, but may be worse in some cases).

In [4]:
from nltk.tokenize import word_tokenize

word_tokenize(texts[0])

['What',
 'is',
 'the',
 'step',
 'by',
 'step',
 'guide',
 'to',
 'invest',
 'in',
 'share',
 'market',
 'in',
 'india',
 '?']


Task Set all texts to lower case and tokenize them.

In [9]:
tokenized_texts = 'do smth'

assert all(isinstance(row, (list, tuple)) for row in tokenized_texts), \
    sent = [word for line in f for word in line.lower().split()]
assert all(all(isinstance(tok, str) for tok in row) for row in tokenized_texts), \
    sent = [word for line in f for word in line.lower().split()]

is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(not is_latin(token) or token.islower() for tokens in tokenized_texts for token in tokens),\
    sent = [word for line in f for word in line.lower().split()]

SyntaxError: ignored

In [7]:
print([' '.join(row) for row in tokenized_texts[:2]])

['d', 'o']



Let's practice a small model on the received texts:

In [8]:
from gensim.models import Word2Vec

model = Word2Vec(tokenized_texts, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

RuntimeError: ignored

## We study the resulting model

Hooray, now you can do the same that was on `rusvectores`.

Get vector for word:

In [10]:
model.get_vector('anything')

NameError: ignored

Find the closest words:

In [11]:
model.most_similar('bread')

NameError: ignored


Or even like this:

In [12]:
model.most_similar(positive=['coder', 'money'], negative=['brain'])

NameError: ignored

And so of course:

In [13]:
model.most_similar([model.get_vector('politician') - model.get_vector('power') + model.get_vector('honesty')])

NameError: ignored


** Task ** Look for analogies yourself.

## Visualize the model


Let's look at the projections of the first thousand of the most frequent words.

In [14]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

NameError: ignored


** Task ** Build a matrix of embeddingings of these words:

In [15]:
word_vectors = model.vectors[[model.vocab[word].index for word in words]]

assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), model.vectors.shape[1])
assert np.isfinite(word_vectors).all()

NameError: ignored

### PCA

The simplest linear dimension reduction method is __P__rincipial __C__omponent __A__nalysis.

The PCA is looking for axes on which the data will have the largest scatter when projected.

! [pca] (https://i.stack.imgur.com/Q7HIP.gif)
* From [https://stats.stackexchange.com/a/140579 ((https://stats.stackexchange.com/a/140579) ()

As a result, you can take the projections onto the first few components - and save as much information as possible, reducing the dimension.

Beautiful visualizations can be found [here] (http://setosa.io/ev/principal-component-analysis/).

** Task ** Use the PCA from sklearn, and then center and normalize the result.

In [16]:
<imports>

def get_pca_projection(word_vectors):
    <code>

SyntaxError: ignored

In [17]:
word_vectors_pca = get_pca_projection(word_vectors)

assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1 - word_vectors_pca.std(0))) < 1e-5, "points must have unit variance"

NameError: ignored

Visualize what happened:

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: 
        pl.show(fig)
    return fig

In [19]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

NameError: ignored

### TSNE

A more interesting and complex (non-linear) method for visualizing high-dimensional spaces is TSNE. You can look at it in detail [here] (https://distill.pub/2016/misread-tsne/) (even more beautiful pictures!).

** Task ** As with PCA, use TSNE from sklearn and normalize + center the result.

In [20]:
from sklearn.manifold import TSNE

def get_tsne_projection(word_vectors):
    <your code>

SyntaxError: ignored

In [21]:
word_tsne = get_tsne_projection(word_vectors)
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

NameError: ignored

## Embedded phrases

Now for tests it will be useful to use fixed embeddings. Let's load already trained model.
(see which models and models are available at all, by calling `api.info ()`)

In [22]:
import gensim.downloader as api

model = api.load('glove-twitter-100')

[==============================================----] 93.7% 362.6/387.1MB downloaded

A simple and cheap way to calculate embedding of a phrase, when there are word embeddings - to average.

So let's do this: tokenize and lower-case the phrase, average the embeddingings of the words for which they are counted.

** Task ** Write a function to count phrase embedding.

In [23]:
def get_phrase_embedding(model, phrase):    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    <write me>
    
    return vector

SyntaxError: ignored

In [24]:
vector = get_phrase_embedding(model, "I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.30757686, -0.05861897,  0.143751  , -0.11104885, -0.96929336,
                             -0.21928601,  0.21652265,  0.14978765,  1.4842536 ,  0.017826  ],
                              dtype=np.float32))

NameError: ignored

Calculate the vector of all phrases.

In [25]:
text_vectors = np.array([get_phrase_embedding(model, phrase) for phrase in tokenized_texts])

NameError: ignored


And learn to look for coming!

** Assignment ** Find the k nearest questions by line.

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

def find_nearest(model, text_vectors, texts, query, k=10):
    <write me, please>

SyntaxError: ignored

In [27]:
results = find_nearest(model, text_vectors, texts, query="How do i enter the matrix?", k=10)

print('\n'.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[1] == 'How do I get to the dark web?'
assert results[4] == 'What can I do to save the world?'

NameError: ignored

In [28]:
find_nearest(model, text_vectors, texts, query="How does Trump?", k=10)

NameError: ignored

In [29]:
find_nearest(model, text_vectors, texts, query="Why don't i ask a question myself?", k=10)

NameError: ignored

## Getting a classification

### Bag-of-words

** Task ** Let's collect tokens for the beginning.

In [0]:
tokenized_question1 = ...
tokenized_question2 = ...


** Task ** Calculate cosine proximity between questions.

The easiest way is to implement the function of its calculation by hand:
$$\text{cosine_similarity}(x, y) = \frac{x^{T} y}{||x||\cdot ||y||}$$

In [31]:
<do smth>

cosine_similarities = <and calc the result>

SyntaxError: ignored


** Task ** Calculate cosine proximity between questions.

The easiest way is to implement the function of its calculation by hand:

In [32]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(cosine_similarities, threshold):
    return <accuracy>

thresholds = np.linspace(0, 1, 100, endpoint=False)
plt.plot(thresholds, [accuracy(cosine_similarities, th) for th in thresholds])

SyntaxError: ignored


And run the optimization:

In [33]:
from scipy.optimize import minimize_scalar

res = minimize_scalar(lambda x: -accuracy(cosine_similarities, x), bounds=(0.5, 0.99), method='bounded')
best_threshold = res.x
print('Threshold = {:.5f}, Accuracy = {:.2%}'.format(best_threshold, accuracy(cosine_similarities, best_threshold)))

NameError: ignored

### Tf-idf weights


Instead of stupidly averaging the vectors, you can average them based on the weights - the familiar tf-idf will help in this.

** Task ** Calculate weighted vector of questions. Use `TfidfVectorizer`

`TfidfVectorizer` returns the matrix` (samples_count, words_count) `. And our embeddings have the dimension `(words_count, embedding_dim)`. So you can just multiply them. Then each phrase - the sequence of words $ w_1, \ ldots, w_k $ - is transformed into the vector $ \ sum_i \ text {idf} (w_i) \ cdot \ text {embedding} (w_i) $. This vector should probably be normalized to the number of words $ k $.

** Task ** In addition to tf-idf, you can add filtering of stop words and punctuation.
Stop words can be taken from `nltk`:
```python
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
```

In [34]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

<calculate tf-idf weighted vectors>

SyntaxError: ignored


Let's see what happens after filtering and vectoring:

In [35]:
for col in tfidf_question1[0].tocoo().col:
    print(model.index2word[col], end=' ')

print('\n' + ' '.join(tokenized_question1[0]))

NameError: ignored


** Task ** Calculate quality with new vectors.

In [36]:
cosine_similarities = <calc it somehow>

res = minimize_scalar(lambda x: -accuracy(cosine_similarities, x), bounds=(0.8, 0.99), method='bounded')
best_threshold = res.x
print('Threshold = {:.5f}, Accuracy = {:.2%}'.format(best_threshold, accuracy(cosine_similarities, best_threshold)))

SyntaxError: ignored


## Let's look inside learning word embeddings


The key idea is that a word can be defined by the context in which it occurs:
! [contexts] (https://image.ibb.co/mnQ2uz/2018_09_17_21_07_08.png)
* From [cs224n, Lecture 2] (http://web.stanford.edu/class/cs224n/lectures/lecture2.pdf) *

Watch how everything learns, we will be here: [https://ronxin.github.io/wevi/[ (http: //ronxin.github.io/wevi/).


## Sewing down a word for machine translation!

In [37]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

    100% |████████████████████████████████| 993kB 18.6MB/s 
Archive:  cc.ru.300.vec.zip
  inflating: cc.ru.300.vec           
Archive:  cc.uk.300.vec.zip
  inflating: cc.uk.300.vec           


Let's write a simple implementation of the machine translation model.

The idea is based on the article [Word Translation Without Parallel Data] (https://arxiv.org/pdf/1710.04087.pdf). The authors of the repository still have a lot of interesting things: [https://github.com/facebookresearch/MUSE_ ((https://github.com/facebookresearch/MUSE).

And we will translate from Ukrainian to Russian.

! [] (https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)
* blue kit * vs. * blue whale *

In [0]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")


Let's look at a couple of serpeni-august (being a translation)

In [39]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [40]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [41]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Недопустимость', 0.24435284733772278),
 ('конструктивность', 0.23293080925941467),
 ('офор', 0.23256804049015045),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003726661205292),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796566247940063)]

In [0]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")


### Learning mapping from one embedding space to another

We have pairs of words that match each other, and their embedding. Find a transformation from one space to another, in order to bring the words we know to us:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$


This function is very similar to linear regression (without bias).

** Task ** Implement it - use `LinearRegression` from sklearn with` fit_intercept = False`:

In [0]:
mapping = ...

Check where the serpen will go:

In [43]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

NameError: ignored

It should turn out that the top contains different months, but August is not the first.

We will measure percision top-k with k = 1, 5, 10.

** Task ** Implement the following function:

In [44]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(ru_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        <write code here>
    precision_val = num_matches / len(pairs)
    return precision_val

SyntaxError: ignored

In [45]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

NameError: ignored

In [46]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

NameError: ignored

In [47]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

NameError: ignored

### Improving Mapping

It can be shown that mapping is better to build orthogonal:
$$ W ^ * = \ arg \ min_W || WX - Y || _F \ text {, where:} W ^ TW = I $$

You can search for it through SVD:
$$ X ^ TY = U \ Sigma V ^ T \ text {, singular value decompostion} $$

$$ W ^ * = UV ^ T $$

** Task ** Implement this feature.

In [48]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    <write code there>

SyntaxError: ignored

In [49]:
W = learn_transform(X_train, Y_train)

NameError: ignored

In [50]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

NameError: ignored

In [51]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

NameError: ignored


### Writing a translator

We implement a simple word-by-word translator - for each word we will look for its nearest neighbor in the common embedding space. If the word is not in the inserts - just copy it.

In [52]:
with open("fairy_tale.txt", "r") as in f:
    uk_sentences = [line.rstrip().lower() for line in in f]

SyntaxError: ignored

In [53]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    <implement it!>

SyntaxError: ignored

In [54]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

NameError: ignored

In [55]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

NameError: ignored


# Additional materials

## Read
### Base:
[On word embeddings - Part 1, Sebastian Ruder] (http://ruder.io/word-embeddings-1/)
[Deep Learning, NLP, Christosher Olah] (http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)

### How to cluster meanings of multi-valued words:
[Making Sense of Word Embeddings (2016), Pelevina et al] (http://anthology.aclweb.org/W16-1620)

### How to evaluate embeddings
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel] (http://www.aclweb.org/anthology/D15-1036)
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu] (https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)
[Problems With Word Of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui]
[Improving Reliability of Word Task and Performance Measurement (2016), Oded Avraham, Yoav Goldberg] (https://arxiv.org/pdf/1611.03641.pdf)
[Evaluating Word Embeddings Using the Representative Suite of Practical Tasks (2016), N. Nayak] (https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)


## show
[Word Vector Representations: word2vec, Lecture 2, cs224n] (https://www.youtube.com/watch?v=ERibwqs9p38)